In [1]:
!pip install pandas==1.5.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 85.6 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.4
    Uninstalling pandas-2.1.4:
      Successfully uninstalled pandas-2.1.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.1.5 which is incompatible.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
import pickle

In [4]:
df = pd.DataFrame()
with open('/kaggle/input/zero-shot-ner-datasets/Zero-ShotNER/datasets/CDR/CDR_protokol4.pkl', 'rb') as f:
    df = df.append(pickle.load(f))
with open('/kaggle/input/zero-shot-ner-datasets/Zero-ShotNER/datasets/NCBI/NCBI_protokol4.pkl', 'rb') as f:
    df = df.append(pickle.load(f))
df = df.reset_index(drop = True)

In [5]:
df

,ids,cor,class,text,labels
0,227508,CDR,[Chemical],"[Naloxone, reverses, the, antihypertensive, ef...","[1, 0, 0, 0, 0, 0, 1, 0]"
1,227508,CDR,[Chemical],"[In, unanesthetized, ,, spontaneously, hyperte...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,227508,CDR,[Chemical],"[2, to, 2, mg, /, kg, .]","[0, 0, 0, 0, 0, 0, 0]"
3,227508,CDR,[Chemical],"[The, hypotensive, effect, of, 100, mg, /, kg,...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, ..."
4,227508,CDR,[Chemical],"[Naloxone, alone, did, not, affect, either, bl...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...
51674,9949197,NCBI,"[Specific, Disease]","[Although, emerin, was, abundant, in, the, mem...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
51675,9949197,NCBI,"[Specific, Disease]","[This, distribution, of, emerin, was, similar,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
51676,9949197,NCBI,"[Specific, Disease]","[In, contrast, ,, lamin, B1, was, absent, from...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
51677,9949197,NCBI,"[Specific, Disease]","[Lamin, B1, is, also, almost, completely, abse...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [6]:
df['labels_sum'] = df.labels.apply(lambda lst: sum(lst))
df['klasa'] = df["class"].apply(lambda lst: ' '.join(lst))
df_hasones = df[df.labels_sum!=0]
df_onlyzeros = df[df.labels_sum==0]

In [7]:
df_hasones_train = pd.DataFrame()
for i in df_hasones.klasa.unique():
    df_hasones_train = df_hasones_train.append(df_hasones[df_hasones.klasa == i].sample(frac = 0.85))

df_onlyzeros_train = pd.DataFrame()
for i in df_onlyzeros.klasa.unique():
    df_onlyzeros_train = df_onlyzeros_train.append(df_onlyzeros[df_onlyzeros.klasa == i].sample(frac = 0.85))
df_hasones_rest = df_hasones.drop(df_hasones_train.index)
df_onlyzeros_rest = df_onlyzeros.drop(df_onlyzeros_train.index)
df_hasones_valid = pd.DataFrame()
for i in df_hasones_rest.klasa.unique():
    df_hasones_valid = df_hasones_valid.append(df_hasones_rest[df_hasones_rest.klasa == i].sample(frac = 0.33))

df_onlyzeros_valid = pd.DataFrame()
for i in df_onlyzeros_rest.klasa.unique():
    df_onlyzeros_valid = df_onlyzeros_valid.append(df_onlyzeros_rest[df_onlyzeros_rest.klasa == i].sample(frac = 0.33))
df_hasones_test = df_hasones_rest.drop(df_hasones_valid.index)
df_onlyzeros_test = df_onlyzeros_rest.drop(df_onlyzeros_valid.index)
df_train = pd.concat([df_hasones_train, df_onlyzeros_train])
df_valid = pd.concat([df_hasones_valid, df_onlyzeros_valid])
df_test = pd.concat([df_hasones_test, df_onlyzeros_test])
print(df_onlyzeros_test.shape[0] + df_onlyzeros_valid.shape[0] + df_onlyzeros_train.shape[0] == df_onlyzeros.shape[0])
print(df_hasones_test.shape[0] + df_hasones_valid.shape[0] + df_hasones_train.shape[0] == df_hasones.shape[0])
print(df_test.shape[0] + df_valid.shape[0] + df_train.shape[0] == df.shape[0])
print(df_test.shape[0] / df.shape[0])
print(df_valid.shape[0] / df.shape[0])
print(df_train.shape[0] / df.shape[0])

True
True
True
0.10044699007333734
0.049555912459606416
0.8499970974670562


In [8]:
df_train.drop(['labels_sum'], inplace=True, axis=1)
df_train = df_train.reset_index(drop = True)

In [9]:
df_valid.drop(['labels_sum'], inplace=True, axis=1)
df_valid = df_valid.reset_index(drop = True)

In [10]:
df_test.drop(['labels_sum'], inplace=True, axis=1)
df_test = df_test.reset_index(drop = True)

In [11]:
import os
folder_path = "/kaggle/working/Datasets"

if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print("Folder 'Datasets' created successfully.")
else:
    print("Folder 'Datasets' already exists.")

Folder 'Datasets' created successfully.


In [12]:
del df

In [13]:
import gc
gc.collect()

0

In [14]:
import pandas as pd
import pickle
import numpy as np

import os

import torch
from torch.utils.data import Subset

import transformers
from transformers import AutoTokenizer

In [15]:
model_name='dmis-lab/biobert-v1.1'
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [16]:
import os
folder_path = "/kaggle/working/Datasets2"

if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print("Folder 'Datasets2' created successfully.")
else:
    print("Folder 'Datasets2' already exists.")

Folder 'Datasets2' created successfully.


In [17]:
def dump_to_pickle(data_set, file_name, class_name=None):
    if class_name == None:
        folders = os.path.join('Datasets2')
    else:
        folders = os.path.join('Datasets2', class_name)
    os.makedirs(folders, exist_ok=True)
    filename = file_name+'.pkl'
    file_path = os.path.join(folders, filename)
    outfile = open(file_path,'wb')
    pickle.dump(data_set,outfile, protocol=4)

In [18]:
def BERTtokenizer_ClassText(df):

    tokenized_encodings = tokenizer(df["class"].to_list(),
                                 df["text"].to_list(),
                                 truncation=True,
                                 is_split_into_words=True,
                                 add_special_tokens=True,
                                 padding='max_length',
                                 max_length=512)
    return tokenized_encodings

In [19]:
def label_m(df, tokenized_encodings, label_all_tokens = True):

    labels = list()
    for i, label in enumerate(df['labels']):
        word_ids = tokenized_encodings.word_ids(batch_index=i)  
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                if word_idx < len(label):
                    label_ids.append(label[word_idx])
                else:
                    label_ids.append(-100)
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        stop = label_ids.index(-100, 2) 
        label_ids = label_ids[:1] + [1 for x in label_ids[1:stop]] + label_ids[stop:]
        labels.append(label_ids)
    return labels

In [20]:
class Dataset_maker(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [21]:
tokenized_encodings_train = BERTtokenizer_ClassText(df_train)
labels_train = label_m(df_train, tokenized_encodings_train, label_all_tokens = True)
dataset_train = Dataset_maker(tokenized_encodings_train, labels_train)

In [22]:
del tokenized_encodings_train

In [23]:
del labels_train

In [24]:
tokenized_encodings_valid = BERTtokenizer_ClassText(df_valid)
labels_valid = label_m(df_valid, tokenized_encodings_valid, label_all_tokens = True)
dataset_valid = Dataset_maker(tokenized_encodings_valid, labels_valid)


In [25]:
del tokenized_encodings_valid
del labels_valid

In [26]:
tokenized_encodings_test = BERTtokenizer_ClassText(df_test)
labels_test = label_m(df_test, tokenized_encodings_test, label_all_tokens = True)
dataset_test = Dataset_maker(tokenized_encodings_test, labels_test)

In [27]:
del tokenized_encodings_test

In [28]:
del labels_test

In [29]:
gc.collect()

0

In [30]:
gc.collect()

0

In [31]:
import pandas as pd
import pickle
import numpy as np

import os

import torch
from torch.utils.data import Subset

import transformers
from transformers import AutoTokenizer
from transformers import BertForTokenClassification, Trainer, TrainingArguments

import random
import time

2024-04-27 17:43:35.814198: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-27 17:43:35.814289: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-27 17:43:35.952323: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [32]:

class P_Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

def load_from_pickle(pickle_name):
    with open(pickle_name, 'rb') as fh:
        unpickled_object = pickle.load(fh)
    return unpickled_object


def dump_to_pickle(data_set, file_name, class_name=None):
    if class_name == None:
        folders = os.path.join('Datasets3')
    else:
        folders = os.path.join('Datasets3', class_name)
    os.makedirs(folders, exist_ok=True)
    filename = file_name+'.pkl'
    file_path = os.path.join(folders, filename)
    outfile = open(file_path,'wb')
    pickle.dump(data_set,outfile, protocol=4)

In [33]:
folder_path = "/kaggle/working/Datasets3"

if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print("Folder 'Datasets3' created successfully.")
else:
    print("Folder 'Datasets3' already exists.")

Folder 'Datasets3' created successfully.


In [34]:
def Dataset_z(class_name):
    indexes_train = list(df_train.index[df_train['klasa']!=class_name])
    dataset_withoutOne_train = Subset(dataset_train, indexes_train)
    return dataset_withoutOne_train

In [35]:
class_unseen = 'Disease'

In [36]:
train0shot= Dataset_z(class_unseen)

In [37]:
del dataset_train,df_train

In [38]:
gc.collect()

83

In [39]:
folder_path = "/kaggle/working/Results"

# Create the folder if it doesn't exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print("Folder 'Results' created successfully.")
else:
    print("Folder 'Results' already exists.")

Folder 'Results' created successfully.


In [40]:
folder_path = "/kaggle/working/Logs"

# Create the folder if it doesn't exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print("Folder 'Logs' created successfully.")
else:
    print("Folder 'Logs' already exists.")

Folder 'Logs' created successfully.


In [41]:
gc.collect()

0

In [42]:
valid_dataset=dataset_valid

In [43]:
gc.collect()

0

In [44]:
pip install accelerate -U

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.28.0
    Uninstalling accelerate-0.28.0:
      Successfully uninstalled accelerate-0.28.0
Note: you may need to restart the kernel to use updated packages.


In [45]:
pip install transformers[torch]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [46]:
import wandb

In [47]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [48]:
import pandas as pd
import pickle
import numpy as np

import os

import torch
from torch.utils.data import Subset

import transformers
from transformers import AutoTokenizer
from transformers import BertForTokenClassification, Trainer, TrainingArguments

import random
import time

In [49]:
import wandb
from transformers import EarlyStoppingCallback

wandb.login(key='dc8c78166e5a09fdf124196c16f5467b0bdd3e66')

training_args = TrainingArguments(
    output_dir='/content/Results' + class_unseen + 'ZeroShot',
    num_train_epochs=30,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    logging_dir='/content/Logs' + class_unseen + 'ZeroShot',
    save_strategy='epoch',
    evaluation_strategy='epoch',
    load_best_model_at_end=True
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForTokenClassification.from_pretrained(model_name, num_labels=2)
model.to(device)

early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=5,
    early_stopping_threshold=0.0001
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train0shot,
    eval_dataset=valid_dataset,
    callbacks=[early_stopping_callback],
)

start_time = time.time()
trainer.train()
total_time = time.time() - start_time

model_path = os.path.join('Results', class_unseen, 'ZeroShot', 'Model')
os.makedirs(model_path, exist_ok=True)
model.save_pretrained(model_path)

tokenizer_path = os.path.join('Results', class_unseen, 'ZeroShot', 'Tokenizer')
os.makedirs(tokenizer_path, exist_ok=True)
tokenizer.save_pretrained(tokenizer_path)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: Currently logged in as: ishaanb-ib (inlpteam). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.16.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.5
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240427_174419-icy428fe
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run golden-field-64
wandb: ⭐️ View project at https://wandb.ai/inlpteam/huggingface
wandb: 🚀 View run at https://wandb.ai/inlpteam/huggingface/runs/icy428fe/workspace


Epoch,Training Loss,Validation Loss
1,0.035600,0.085172
2,0.020600,0.093150
3,0.011800,0.136792
4,0.009900,0.193200
5,0.007300,0.143474
6,0.004400,0.190010


('Results/Disease/ZeroShot/Tokenizer/tokenizer_config.json',
 'Results/Disease/ZeroShot/Tokenizer/special_tokens_map.json',
 'Results/Disease/ZeroShot/Tokenizer/vocab.txt',
 'Results/Disease/ZeroShot/Tokenizer/added_tokens.json',
 'Results/Disease/ZeroShot/Tokenizer/tokenizer.json')

In [50]:
del train0shot
del valid_dataset

In [51]:
gc.collect()

186

In [52]:
import torch
import numpy as np
import pandas as pd
import pickle

from transformers import Trainer, TrainingArguments, BertForTokenClassification  
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score,recall_score,precision_score

In [53]:
folder_path = "/kaggle/working/DatasetsX"

if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print("Folder 'DatasetsX' created successfully.")
else:
    print("Folder 'DatasetsX' already exists.")

Folder 'DatasetsX' created successfully.


In [54]:
def load_from_pickle(pickle_name):
    with open(pickle_name, 'rb') as fh:
        unpickled_object = pickle.load(fh)
    return unpickled_object   


def dump_to_pickle(data_set, file_name, class_name=None):
    if class_name == None:
        folders = os.path.join('DatasetsX')
    else:
        folders = os.path.join('DatasetsX', class_name)
    os.makedirs(folders, exist_ok=True)
    filename = file_name+'.pkl'
    file_path = os.path.join(folders, filename)
    outfile = open(file_path,'wb')
    pickle.dump(data_set,outfile, protocol=4)

    
class u_Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [55]:

indexes_test = list(df_test.index[df_test['klasa']=='Disease'])
dataset_withOne_test = Subset(dataset_test, indexes_test)


In [56]:
gc.collect()

0

In [57]:
lst_indexes_FewShot_test = list(df_test.index[(df_test['klasa']=='Disease') & (df_test['labels'].apply(lambda lst : sum(lst))>0)])
indexes_UnseenClass_test = random.choices(lst_indexes_FewShot_test, k=1000)
dataset_UnseenClass_test = df_test.loc[indexes_UnseenClass_test]


In [58]:
gc.collect()

0

In [59]:
import torch
import numpy as np
import pandas as pd
import pickle

from transformers import Trainer, TrainingArguments, BertForTokenClassification
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, recall_score, precision_score
def testing1(test_set_path, model_path, df_test_path):
    testset = test_set_path
    df_test = df_test_path

    model = BertForTokenClassification.from_pretrained(model_path, num_labels=2)
    args = TrainingArguments(output_dir='./evaldir', per_device_eval_batch_size=16)

    evaler = Trainer(
        args=args,
        model=model
    )

    pred=evaler.predict(testset)
    test_indexs_new=df_test.index.to_list()
    wids=np.array([testset.dataset.encodings.encodings[ii].word_ids for ii in test_indexs_new])
    wids[wids==None]=-1
    wids=wids.astype(int)

    type_ids=np.array([testset.dataset.encodings.encodings[ii].type_ids for ii in test_indexs_new],dtype=bool)
    pre=pred[0].argmax(axis=-1)
    pre_list=[]
    test_list=[]
    for ii in range(wids.shape[0]):
        test_list.append(wids[ii][type_ids[ii]])
        pre_list.append(pre[ii][type_ids[ii]])

    labels=[]
    for ii in range(len(pre_list)):
        labels.append(np.array(range(test_list[ii].max()+1)))
        for jj in labels[ii]:
            bb=np.where(test_list[ii]==jj)[0]
            labela=np.array(pre_list[ii])[bb].mean()
            if labela>0.01:
                labels[ii][jj]=1
            else:
                labels[ii][jj]=0

    labels_original=list(df_test.iloc[:]['labels'])
    lab=np.array([])       # predicted labels
    labor=np.array([])     # true labels labels
    for ii in range(len(labels)):
        if len(labels[ii])==len(labels_original[ii]):
            lab=np.concatenate((lab,labels[ii]))
            labor=np.concatenate((labor,labels_original[ii]))

    accuracy = accuracy_score(labor, lab, normalize=True)
    precision = precision_score(labor, lab, average='weighted')
    recall = recall_score(labor, lab, average='weighted')
    f1 = f1_score(labor, lab, average='weighted')
    matrix = confusion_matrix(labor, lab)

    # Statistics in a proper tabular form
    statistics = pd.DataFrame({
        'Metric': ['Accuracy', 'Precision', 'Recall', 'F1'],
        'Value': [accuracy, precision, recall, f1]
    })

    # Print out confusion matrix
    print("\nConfusion Matrix:")
    print(matrix)

    return statistics

model_path = '/kaggle/working/Results/Disease/ZeroShot/Model'
test_set_path = dataset_withOne_test
df_test_path = dataset_UnseenClass_test
statistics = testing1(test_set_path, model_path, df_test_path)

# Print out statistics
print("Statistics:")
print(statistics)


Confusion Matrix:
[[17493  3554]
 [ 1972   503]]
Statistics:
      Metric     Value
0   Accuracy  0.765071
1  Precision  0.817175
2     Recall  0.765071
3         F1  0.788933
